In [1]:
import datetime
import Preprocesado
import Clustering
import json
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [2]:
input_events_path = 'data.parquet'
input_contracts_path = 'original_contracts.parquet'
fecha_min = '2019-6-30' #Formato de fecha: año-mes-dia
fecha_max = '2019-7-31' 
max_num_clusters = 10
find_outliers = True
escalado = 'minmax'
grafica = True

In [3]:
pd.set_option('display.max_columns',500)

In [4]:
print('Leyendo Datos')
data = pq.ParquetDataset(input_events_path).read(use_threads = False).to_pandas()
original_contracts = pq.ParquetDataset(input_contracts_path).read(use_threads = False).to_pandas()

Leyendo Datos


In [5]:
data = data.sample(frac=0.1)  #Estas lineas simplemente son para acelerar las corridas. 
original_contracts = original_contracts.sample(frac=0.1)

In [6]:
data['event_date'] = pd.to_datetime(data.event_date, format = '%d-%b-%y %I.%M.%S.%f %p')
bloque = data[data['event_date'] < pd.to_datetime(fecha_max).date()]
bloque = bloque[bloque['event_date'] > pd.to_datetime(fecha_min).date()]
print('Filtrando clientes con al menos un contrato activo')
contracts = Preprocesado.filtra_clientes_activos(original_contracts,bloque)
#Binariza domotic event, que es la unica columna a binarizar
bloque['domotic_event'] = pd.Series(np.where(bloque.domotic_event.fillna(0) != 0, 1, 0))
print('Agrupando Eventos')
structured_events = Preprocesado.estructura_eventos(bloque)
print('Desglosando variables temporales')
frecuencia_eventos = Preprocesado.crea_var_por_intervalo_de_time(bloque)
print('Agrupando variables temporales por cliente y evento')
frec_x_interv = Preprocesado.cuenta_frecuencia_por_intervalo(frecuencia_eventos)
print('Agrupando origen de eventos por cliente')
data_origin = bloque[['client_id','domotic_event','mastermind_event']].groupby('client_id').mean()
print('Agregando variables a Tabla de Contratos')
client_contracts = Preprocesado.agrupaciones_por_cliente(contracts)
print('Uniendo tablas')
tabla_nivel = Preprocesado.merge_dataset(frec_x_interv,data_origin,structured_events,client_contracts)
print('Guardando tabla a nivel')
tabla_nivel.to_csv('TablaANivel.csv',index=False)


/home/mmontero/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
/home/mmontero/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


Filtrando clientes con al menos un contrato activo
Agrupando Eventos
Desglosando variables temporales
Agrupando variables temporales por cliente y evento
Agrupando origen de eventos por cliente
Agregando variables a Tabla de Contratos
Uniendo tablas
Guardando tabla a nivel


In [7]:
tabla_nivel = pd.read_csv('TablaANivel.csv')

In [8]:
tabla_nivel.head()

,client_id,dawn,morning,afternoon,evening,weekday,weekend,early_month,midearly_month,midlate_month,late_month,domotic_event,mastermind_event,Alarma de antifog,Alarma de atraco,Alarma de coaccion,Alarma de fuego,Alarma de inundacion,Alarma de robo,Alarma de sabotaje,Alarma de temperatura,Alarma medica,Apertura,Armado parcial correcto,Armado parcial erroneo,Armado parcial erroneo. Problemas de comunicacion,Armado total correcto,Armado total erroneo,Armado total erroneo. Problemas de comunicacion,Asignado,Baja bateria de termostato,Bateria baja,Cambio Codigo Usuario. Problemas de comunicacion,Cambio de Codigo de Usuario correcto,Completado,Desarmado correcto,Desarmado erroneo,Desarmado erroneo. Problemas de comunicacion,En Instalaci?n,En ruta,Evidencia,Fallo de corriente,Fallo linea comunicacion,Fallo por inhibicion,Im?genes Disponibles,Pedir imagen correcto,Pedir imagen erroneo,Pedir imagen erroneo. Problemas de comunicacion,Restaturacion de fallo de corriente,Restauracion bateria baja,Restauracion fallo de linea de comunicacion,Video Verificacion,codigo_subtipo_local_en_altura,codigo_subtipo_local_other,codigo_subtipo_local_unifamiliar,codigo_regimen_venta,antiguedad,precio_alta_mean,precio_alta_max,precio_alta_sum,id_contrato_count
0,0001530,1,1,0,1,2,1,1,1,0,1,NaN,1.00,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0,735686,149.00,149.00,149.00,1
1,0002096,0,3,9,4,15,1,12,3,1,0,0.0,0.75,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0.0,1.0,0.0,0.0,732972,511.00,511.00,511.00,1
2,0004128,5,12,0,0,13,4,7,6,2,2,0.0,1.00,0,0,0,0,0,3,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,733994,3393.00,3393.00,3393.00,1
3,0004951,0,4,0,1,5,0,4,1,0,0,NaN,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,733891,172.41,172.41,172.41,1
4,0007171,0,1,0,1,2,0,1,1,0,0,0.0,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,1.0,735431,1288.00,1288.00,1288.00,1


In [8]:
print('Filtra los datos de acuerdo al config')
cfg_path = './columnas.config'
data_final = Preprocesado.filtra_por_config(tabla_nivel,cfg_path)


Filtra los datos de acuerdo al config
(10652, 23)


In [27]:
print('Escala los datos')
data_escalada = Preprocesado.escala_datos(escalado,data_final)


Escala los datos


/home/mmontero/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
if find_outliers==True:

    print('Quitamos outliers según algoritmo IsolationForest')
    data_final = Preprocesado.get_outliers_out(data_escalada)

In [ ]:
print('Clusterizando')
km_entrenados = Clustering.clusterize(data_escalada,max_num_clusters)


In [ ]:
print('Generando Tabla de Centroides')
columnas = data_final.set_index('client_id').columns
tablasummary = Clustering.summary_sin_entrenar(km_entrenados,columnas)
display(tablasummary)


In [ ]:
print('Generando Tabla de Clientes mas cercanos al Centroide')
tablaclosest = Clustering.get_closest_to_centroids_sin_entrenar(km_entrenados,data_escalada,data_final)
display(tablaclosest)


In [ ]:
print('Generando graficos de silueta')
Clustering.silhouette_metric_sin_entrenar(km_entrenados,data_escalada)


In [ ]:
print('Clusterización finalizada')

#### FALTA GUARDAR LOS DATOS CLUSTERIZADOS EN ALGUN LADO ####